In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# if run in colab 
!pip install Keras==2.0.9
!pip install cytoolz==0.10.1

import plac
import random
import pathlib
import cytoolz
import numpy
import pandas as pd 
import keras
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.layers import TimeDistributed
from keras.optimizers import Adam
from spacy.compat import pickle
import spacy
# !python -m spacy download en_core_web_sm 


!ls
%cd /content/drive/My Drive/Colab Notebooks/CA_NLP


     |████████████████████████████████| 307kB 2.8MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.9 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
     |████████████████████████████████| 481kB 2.8MB/s 
  Created wheel for cytoolz: filename=cytoolz-0.10.1-cp36-cp36m-linux_x86_64.whl size=1256652 sha256=ce98da95fe759ef05f11d840a70b6c1b9f079cbc797f2565e1ce7aabddae39ff
  Stored in directory: /root/.cache/pip/wheels/64/2a/18/d962b614e055577e7d9a3e4813e0742f822ca9c8800cc3783a
Successfully built cytoolz


Using TensorFlow backend.


drive  sample_data
/content/drive/My Drive/Colab Notebooks/CA_NLP


In [0]:
class SentimentAnalyser(object):
    @classmethod

    # def load(cls, path, nlp, max_length=100):
    def load(cls, nlp, path, max_length):
        with open(path+"config.json") as file_:
            model = model_from_json(file_.read())
        with open(path+"model",'rb') as file_:
            lstm_weights = pickle.load(file_)
        embeddings = nlp.vocab.vectors.data
        model.set_weights([embeddings] + lstm_weights)
        return cls(model, max_length=max_length)

    def __init__(self, model, max_length=100):
            self._model = model
            self.max_length = max_length

    def __call__(self, doc):
            X = get_features([doc], self.max_length)
            y = self._model.predict(X)
            self.set_sentiment(doc, y)

    def pipe(self, docs, batch_size=1000):
            def get_features(docs, max_length):
                    docs = list(docs)
                    Xs = numpy.zeros((len(docs), max_length), dtype="int32")
                    for i, doc in enumerate(docs):
                        j = 0
                        for token in doc:
                            vector_id = token.vocab.vectors.find(key=token.orth)
                            if vector_id >= 0:
                                Xs[i, j] = vector_id
                            else:
                                Xs[i, j] = 0
                            j += 1
                            if j >= max_length:
                                break
                    return Xs
            for minibatch in cytoolz.partition_all(batch_size, docs):
                minibatch = list(minibatch)
                sentences = []
                for doc in minibatch:
                    sentences.extend(doc.sents)
           
                Xs = get_features(sentences, self.max_length)
                ys = self._model.predict(Xs)
                for sent, label in zip(sentences, ys):
                    sent.doc.sentiment += label 
                for doc in minibatch:
                    yield doc



    def set_sentiment(self, doc, y):
            doc.sentiment = float(y[0])
            
    def get_labelled_sentences(docs, doc_labels):
        labels = []
        sentences = []
        for doc, y in zip(docs, doc_labels):
            for sent in doc.sents:
                sentences.append(sent)
                labels.append(y)
        return sentences, numpy.asarray(labels, dtype="int32")

    def compile_lstm(embeddings, shape, settings):
        model = Sequential()
        model.add(
            Embedding(
                embeddings.shape[0],
                embeddings.shape[1],
                input_length=shape["max_length"],
                trainable=False,
                weights=[embeddings],
                mask_zero=True,
            )
        )
        model.add(TimeDistributed(Dense(shape["nr_hidden"], use_bias=False)))
        model.add(
            Bidirectional(
                LSTM(
                    shape["nr_hidden"],
                    recurrent_dropout=settings["dropout"],
                    dropout=settings["dropout"],
                )
            )
        )
        model.add(Dense(shape["nr_class"], activation="sigmoid"))
        model.compile(
            optimizer=Adam(lr=settings["lr"]),
            loss="binary_crossentropy",
            metrics=["accuracy"],
        )
        return model


In [0]:
path_vectors = "./en_vectors_web_lg/en_vectors_web_lg-2.1.0"
path_model = './model_lstm/' 
path_data = './Tripadvisor_data.csv'

# for preprocess
nlp = spacy.load('en_core_web_sm',disable=['parser','ner'])
nlp.add_pipe(nlp.create_pipe("sentencizer"))

# for sentiment
nlp_sentiment = spacy.load(path_vectors)
nlp_sentiment.add_pipe(nlp.create_pipe("sentencizer"))
nlp_sentiment.add_pipe(SentimentAnalyser.load(nlp_sentiment, path_model, 100))

# check pipeline
print(nlp.pipeline)
print(nlp_sentiment.pipeline)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






[('tagger', <spacy.pipeline.pipes.Tagger object at 0x7f405493e390>), ('sentencizer', <spacy.pipeline.pipes.Sentencizer object at 0x7f406295b0f0>)]
[('sentencizer', <spacy.pipeline.pipes.Sentencizer object at 0x7f406295b048>), ('SentimentAnalyser', <__main__.SentimentAnalyser object at 0x7f403dc30fd0>)]
9     1824
30    1452
18    1359
38    1040
31     943
8      916
6      798
36     750
14     698
10     690
13     554
4      507
17     480
29     420
27     408
35     373
37     315
28     312
15     276
34     259
25     255
24     224
3      205
33     202
21     199
7      175
16     164
19     156
32     154
22     135
39     130
1      126
11     122
5      117
26     110
2       82
Name: res

In [0]:
def docs2sent(docs):

# pipeline can process with batches ,will be faster than pipeline single element 
  allsents =list()
  for docsents in list(nlp.pipe(list(docs['review']))):
    for doc in docsents.sents:
      allsents.append(doc.text)
  return allsents

def num_polarity(df):

  num_pos = sum(df['polarity'])
  num_neg = len(df['polarity']) - num_pos
  print('number of positive reviews: ' + str(num_pos))
  print('number of negative reviews: ' + str(num_neg))

  return num_pos, num_neg

def sent2score(df):

  allsents = docs2sent(df)
  review =list()
  score = list()
  for doc in nlp_sentiment.pipe(allsents):
      review.append(doc.text)
  main = pd.DataFrame({'review':review})
  return main

def get_taste(res_food,res_name,main):
  food_graph = res_food[res_food['name1']==res_name]['name2'].tolist()
  matcher = PhraseMatcher(nlp_food.vocab, attr="LOWER")
  patterns = [nlp_food.make_doc(name) for name in food_graph]
  matcher.add("Names", None, *patterns)
  index = list()
  food_entity = list()
  food_taste = list()
  for idx, val in enumerate(main['review']):
    doc = nlp_food(val)
    for token in doc:
      if token.pos_ in ['ADJ'] and token.text in tastelist:
        for match_id, start, end in matcher(doc):
            index.append(idx)
            food_taste.append(token.text)
            food_entity.append(doc[start:end])
            df = pd.DataFrame({'index':index,'food_entity':food_entity,'food_taste':food_taste})

  df['food_entity'] = df['food_entity'].astype(str) 
  return df

In [0]:
# prepare
from spacy.matcher import PhraseMatcher
nlp_food = spacy.load('en_core_web_sm',disable=['parser','ner'])
res_food = pd.read_excel('RelData.xlsx',sheet_name='res_food')
res_food_2 = pd.read_excel('RelData.xlsx',sheet_name='food_res')
tastelist = ['sour','sweet','bitter','hot','spicy','salty','soft','tender',
             'hard','crispy','fresh','curry']
trip = pd.read_csv(path_data,
                   usecols=['res_id','review'],
                   header=0,
                   encoding='utf-8')

all_name = res_food['name1'].drop_duplicates().reset_index().drop(columns='index').drop(index=[0,15,25])['name1'].tolist()
all_id = [1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,21,22,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38]
graph = pd.DataFrame()


for res_name,res_id in zip(all_name,all_id):
  
  trip_df = trip[trip['res_id']==res_id]   # map res_id 
  print(res_name,res_id,len(trip_df))
  trip_df['review'] = trip_df['review'].str.lower()
  main = sent2score(trip_df)
  print(len(main))
  df = get_taste(res_food,res_name,main)
  taste_group = df.groupby(by=['food_entity','food_taste']).count().sort_values(by=['food_entity','index'],ascending=False).groupby(level=0).head(1).drop(columns='index').reset_index()
  new_df = res_food_2[res_food_2['name2']==res_name].merge(right=taste_group,left_on='name1',how='left',right_on='food_entity')
  taste_graph = new_df.drop(columns=['taste','food_entity']).rename(columns={'food_taste':'taste'})
  print(taste_graph)
  graph = graph.append(taste_graph)

graph.to_csv('taste_foodgraph.csv')

Le Noir Bar & Lounge 1 126


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


465
      name1 relationship                 name2 taste
0  cocktail        exist  Le Noir Bar & Lounge   NaN
1     pizza        exist  Le Noir Bar & Lounge   NaN
2      beer        exist  Le Noir Bar & Lounge   NaN
3   whiskey        exist  Le Noir Bar & Lounge  sour
Marmaris Restaurant 2 82
296
               name1 relationship                name2   taste
0            chicken        exist  Marmaris Restaurant   fresh
1         mandi rice        exist  Marmaris Restaurant  tender
2        layer juice        exist  Marmaris Restaurant     NaN
3  saudi kabsah rice        exist  Marmaris Restaurant     NaN
4        mango juice        exist  Marmaris Restaurant     NaN
5          shawarmas        exist  Marmaris Restaurant     NaN
6            briyani        exist  Marmaris Restaurant     NaN
Fu Lin Men (CSC) 3 205
1012
                  name1 relationship             name2  taste
0          chicken soup        exist  Fu Lin Men (CSC)  fresh
1               chicken        exist  Fu Lin M

In [0]:
graph.to_excel('taste_foodgraph.xlsx',index=False)